# 🔪JAX锋芒毕露🔪

> 作者：levskaya@ mattjj@
> 
> 在意大利的乡间漫步时，人们会毫不犹豫地告诉您JAX具有：“una anima di pura programmazione funzionale（纯函数式编程的灵魂）”

JAX是一种用于表达和转换组合的数值程序。JAX还能够便于用于CPU或加速器（GPU或TPU）。JAX对于许多数值和科学编程都非常有用，但前提是它们是在一下描述的某些约束条件下编写而成的。



In [1]:
import numpy as np
from jax import grad, jit
from jax import lax
from jax import random
import jax 
import jax.numpy as jnp
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import rcParams

rcParams['image.interpolation'] = 'nearest'
rcParams['image.cmap'] = 'viridis'
rcParams['axes.grid'] = False

# 🔪纯函数

JAX变换和编译仅适用于功能纯净的Python函数：所有输入数据均通过函数参数传递，所有结果均通过函数结果输出。如果使用相同的输入调用纯函数，则总会返回相同的结果。

这是一些功能上并非纯函数的示例，对于这些函数，JAX的行为不同于Python解释器。注意，JAX系统不能保证这些行为。使用JAX的正确方法是使用功能纯粹的纯Python函数。

In [2]:
def impure_print_side_effect(x):
    print("Executing function") # 这就是一种副作用
    return x

# The side-effects appear during the first run  
# 在第一次执行时就出现副作用
print("First call: ", jit(impure_print_side_effect)(4.))

# Subsequent runs with parameters of same type and shape may not show the side-effect
# This is because JAX now invokes a cached compilation of the function
# 使用相同类型和形状的参数进行的后续运行可能不会显示副作用
# 这是因为JAX现在调用了该函数的缓存编译
print("Second call: ", jit(impure_print_side_effect)(5.))

# JAX re-runs the Python function when the type or shape of the argument changes
# 当参数的类型或者形状更改时，JAX重新运行Python函数
print("Third call, different type: ", jit(impure_print_side_effect)(jnp.array([5.])))


Executing function
First call:  4.0
Second call:  5.0
Executing function
Third call, different type:  [5.]


In [3]:
g = 0.
def impure_uses_globals(x):
  return x + g

# JAX captures the value of the global during the first run
# JAX在第一次执行时捕捉到全局变量
print ("First call: ", jit(impure_uses_globals)(4.))

g = 10.  # 更新全局变量

# Subsequent runs may silently use the cached value of the globals
# 以下的结果将会默认使用缓存的全局变量
print ("Second call: ", jit(impure_uses_globals)(5.))

# JAX re-runs the Python function when the type or shape of the argument changes
# This will end up reading the latest value of the global
# 参数的类型或者形状更改时，JAX重新运行Python函数
# 这样将会重新读取最新的全局变量值
print ("Third call, different type: ", jit(impure_uses_globals)(jnp.array([4.])))

First call:  4.0
Second call:  5.0
Third call, different type:  [14.]


In [4]:
g = 0.
def impure_saves_global(x):
  global g
  g = x
  return x

# JAX runs once the transformed function with special Traced values for arguments
# JAX运行带有参数的特殊跟踪值转换后的函数
print ("First call: ", jit(impure_saves_global)(4.))
print ("Saved global: ", g)  # 保存的全局变量带有内部的JAX值

First call:  4.0
Saved global:  Traced<ShapedArray(float64[], weak_type=True)>with<DynamicJaxprTrace(level=0/1)>


即时Python函数实际上在内部使用有状态的对象，只要它不读取或写入外部状态，它在功能上也可以是纯函数：

In [5]:
def pure_uses_internal_state(x):
    state = dict(even=0, odd=0)
    for i in range(10):
        state['even' if i % 2 == 0 else 'odd'] += x
    return state['even'] + state['odd']

print(jit(pure_uses_internal_state)(5.))

50.0


不建议在需要 `jit` 的任何JAX函数或任何控制流原语中使用迭代器。因为迭代器是一个Python对象，它引入状态以检测下一个元素。因此，它与JAX功能编程模型不兼容。下面的代码中有一些将迭代器与JAX一起使用的错误示例。它们中的大多数返回错误，但有些会产生令人意外的结果：

In [6]:
import jax.numpy as jnp
import jax.lax as lax
from jax import make_jaxpr

# lax.fori_loop
array = jnp.arange(10)
print(lax.fori_loop(0, 10, lambda i, x: x+array[i], 0)) # 预期结果是45
iterator = iter(range(10))
print(lax.fori_loop(0, 10, lambda i, x:x+next(iterator), 0)) #意外结果是0

#lax.scan
def func11(arr, extra):
    ones = jnp.ones(arr.shape)
    def body(carry, aelems):
        ae1, ae2 = aelems
        return (carry + ae1 * ae2 + extra, carry)
    return lax.scan(body, 0., (arr, ones))
make_jaxpr(func11)(jnp.arange(16), 5.)

#lax.cond
array_operand = jnp.array([0.])
lax.cond(True, lambda x: x+1, lambda x:x-1, array_operand)
iter_operand = iter(range(10))
# 报错
lax.cond(True, lambda x: next(x)+1, lambda x:next(x)-1, iter_operand)

45
0


TypeError: Value <range_iterator object at 0x7fc0143885d0> with type <class 'range_iterator'> is not a valid JAX type

## 🔪 就地更新

在Numpy中我们习惯这么做：

In [7]:
numpy_array = np.zeros((3, 3), dtype=np.float32)
print("original array:")
print(numpy_array)

# In place, mutating update
# 直接可变更新
numpy_array[1, :] = 1.0
print("updated array:")
print(numpy_array)

original array:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
updated array:
[[0. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]


但是，如果尝试就地更新JAX数组，则会收到错误消息！ （☉_☉）

In [8]:
jax_array = jnp.zeros((3,3), dtype=jnp.float32)

# In place update of JAX's array will yield an error!
# 就地更新JAX数组则会导致错误
try:
  jax_array[1, :] = 1.0
except Exception as e:
  print("Exception {}".format(e))

Exception '<class 'jax.interpreters.xla._DeviceArray'>' object does not support item assignment. JAX arrays are immutable; perhaps you want jax.ops.index_update or jax.ops.index_add instead?


### 原因是啥？！

允许就地改变变量使得程序分析和转换非常困难。JAX需要数值程序的纯函数表达式。

JAX提供了几个函数式的更新函数：[index_update](https://jax.readthedocs.io/en/latest/_autosummary/jax.ops.index_update.html#jax.ops.index_update), [index_add](https://jax.readthedocs.io/en/latest/_autosummary/jax.ops.index_add.html#jax.ops.index_add), [index_min](https://jax.readthedocs.io/en/latest/_autosummary/jax.ops.index_max.html#jax.ops.index_min), [index_max](https://jax.readthedocs.io/en/latest/_autosummary/jax.ops.index_max.html#jax.ops.index_max), 以及 [index](https://jax.readthedocs.io/en/latest/_autosummary/jax.ops.index.html#jax.ops.index)辅助函数。

️⚠️ 在`jit`代码的 `lax.while_loop`或 `lax.fori_loop`中，切片的大小不能做为参数值的函数，而只能是参数形状的函数——切片开始索引没有这种限制。有关此限制的更多信息，请参见下面的控制流部分。

In [9]:
from jax.ops import index, index_add, index_update

### index_update

如果 **index_update**的**input values** 没有被重用，`jit`编译的代码将会就地执行这些操作：

In [10]:
jax_array = jnp.zeros((3, 3))
print("original array:")
print(jax_array)

new_jax_array = index_update(jax_array, index[1, :], 1.)

print("old array unchanged:")
print(jax_array)

print("new array:")
print(new_jax_array)

original array:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
old array unchanged:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
new array:
[[0. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]


### index_add

如果 **index_add**的**input values** 没有被重用，`jit`编译的代码将会就地执行这些操作：


In [11]:
print("original array:")
jax_array = jnp.ones((5, 6))
print(jax_array)

new_jax_array = index_add(jax_array, index[::2, 3:], 7)
print("new array post-addition:")
print(new_jax_array)

original array:
[[1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]]
new array post-addition:
[[1. 1. 1. 8. 8. 8.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 8. 8. 8.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 8. 8. 8.]]


## 🔪 越界索引

在Numpy中，当您在数组的边界之外索引时往往会报错，如下所示：


In [12]:
try:
  np.arange(10)[11]
except Exception as e:
  print("Exception {}".format(e))


Exception index 11 is out of bounds for axis 0 with size 10


然而，在加速设备上引发错误更加困难。因此，JAX不会报错，而是将索引限制在数组的边界上，也就是返回数组的最后一个值：

In [13]:
jnp.arange(10)[11]


DeviceArray(9, dtype=int64)

注意，由于这种行为，`jnp.nanargmin` 和 `jnp.nanargmax` 对于由NaN组成的切片返回-1，Numpy则会报错。

## 🔪 随机数

> 如果所有由于 `rand()` 不好而导致结果令人怀疑的科学论文都从图书馆的书架上消失了，那么每个书架上的空隙会有拳头那么大。
>
>  —— 数字食谱

### RNGs（随机数生成器）和状态

你已经习惯了使用Numpy或其他库中具有状态的为随机数生成器(PRNGs)，这些函数有助于隐藏很多细节，给您直接准备好伪随机源：



In [14]:
print(np.random.random())
print(np.random.random())
print(np.random.random())

0.24890881647150354
0.5079150611740478
0.013760204455776082


在后端，Numpy使用`Mersenne Twister PRNG`作为其伪随机函数的发动机。PRNG的周期为$2^{19937} - 1$，并且在任何时候都可以用624个32位无符号整数和一个指示该“熵”已用完多少的位置来描述。

In [15]:
np.random.seed(0)
rng_state = np.random.get_state()
print(rng_state)

('MT19937', array([         0,          1, 1812433255, 1900727105, 1208447044,
       2481403966, 4042607538,  337614300, 3232553940, 1018809052,
       3202401494, 1775180719, 3192392114,  594215549,  184016991,
        829906058,  610491522, 3879932251, 3139825610,  297902587,
       4075895579, 2943625357, 3530655617, 1423771745, 2135928312,
       2891506774, 1066338622,  135451537,  933040465, 2759011858,
       2273819758, 3545703099, 2516396728, 1272276355, 3172048492,
       3267256201, 2332199830, 1975469449,  392443598, 1132453229,
       2900699076, 1998300999, 3847713992,  512669506, 1227792182,
       1629110240,  112303347, 2142631694, 3647635483, 1715036585,
       2508091258, 1355887243, 1884998310, 3906360088,  952450269,
       3647883368, 3962623343, 3077504981, 2023096077, 3791588343,
       3937487744, 3455116780, 1218485897, 1374508007, 2815569918,
       1367263917,  472908318, 2263147545, 1461547499, 4126813079,
       2383504810,   64750479, 2963140275, 1709368

每当需要一个随机数时，伪随机状态向量都会在后台自动更新，从而“消耗”了 `Mersenne Twister`中的2个无符号32位整数。

In [16]:
_ = np.random.uniform()
rng_state = np.random.get_state()
print(rng_state)

('MT19937', array([2443250962, 1093594115, 1878467924, 2709361018, 1101979660,
       3904844661,  676747479, 2085143622, 1056793272, 3812477442,
       2168787041,  275552121, 2696932952, 3432054210, 1657102335,
       3518946594,  962584079, 1051271004, 3806145045, 1414436097,
       2032348584, 1661738718, 1116708477, 2562755208, 3176189976,
        696824676, 2399811678, 3992505346,  569184356, 2626558620,
        136797809, 4273176064,  296167901, 3430730584,  331909803,
       1908676996, 1950065095,  604298543, 3615988338, 1570232852,
       1028209748, 1511467721, 2411887154, 4210753555, 3096762720,
        423429618,  659966766, 2937509307, 2222847265,  378636552,
       1142109618, 2509241601, 1521729757,  888533219,  250885260,
       2455816244, 4046047811, 1947467789, 1395351953, 2388948566,
        934627940,  194642258, 1429256273, 2139959677, 1543740405,
       1569613451, 4061840539, 2075690423,  824532376,  844152077,
       3218002536,  897315311,  823414659, 1007534

In [17]:
# Let's exhaust the entropy in this PRNG statevector
# 我们用尽PRNG状态向量中的熵
for i in range(311):
  _ = np.random.uniform()
rng_state = np.random.get_state()
print(rng_state)

('MT19937', array([2443250962, 1093594115, 1878467924, 2709361018, 1101979660,
       3904844661,  676747479, 2085143622, 1056793272, 3812477442,
       2168787041,  275552121, 2696932952, 3432054210, 1657102335,
       3518946594,  962584079, 1051271004, 3806145045, 1414436097,
       2032348584, 1661738718, 1116708477, 2562755208, 3176189976,
        696824676, 2399811678, 3992505346,  569184356, 2626558620,
        136797809, 4273176064,  296167901, 3430730584,  331909803,
       1908676996, 1950065095,  604298543, 3615988338, 1570232852,
       1028209748, 1511467721, 2411887154, 4210753555, 3096762720,
        423429618,  659966766, 2937509307, 2222847265,  378636552,
       1142109618, 2509241601, 1521729757,  888533219,  250885260,
       2455816244, 4046047811, 1947467789, 1395351953, 2388948566,
        934627940,  194642258, 1429256273, 2139959677, 1543740405,
       1569613451, 4061840539, 2075690423,  824532376,  844152077,
       3218002536,  897315311,  823414659, 1007534

In [18]:
# Next call iterates the RNG state for a new batch of fake "entropy".
# 下一步调用为新一批伪造的“熵”迭代RNG状态.
_ = np.random.uniform()
rng_state = np.random.get_state()
print(rng_state)

('MT19937', array([1499117434, 2949980591, 2242547484, 1470907986,   68004624,
        613504879, 2170701638, 3606168244, 1313189820, 2904302179,
       3340054280, 2800779156, 3718152353, 1082918459, 1748036786,
       3125556887, 1246967947, 2050301915, 3440863170, 2306625137,
       2391836667, 1253663658, 2419038162, 3499839328, 3576356820,
       3828856986,  723946277, 1516277410, 1749873187, 2585175776,
       2103116091, 3761404950, 2177145536, 2190050649, 2604636580,
       1049507822, 3538272245, 2566586914, 3538170909, 4282737256,
       3260797503, 2387454175, 2226689230, 2256270485,  436199026,
       1447928333, 1300475185, 3910190296, 2621047601, 2432253395,
       3548512997, 3038311477, 3870448599, 4184179771,  331186464,
       1513235983, 1123184249, 1412176674,  974731669, 1184859182,
       3903198916, 1010728009, 1157972564, 1456817460, 4280740152,
       3287444695, 3162962129, 2065442163,  702491398, 2129714181,
       1271816637, 1310830189, 1626731654, 1866514

神奇的PRNG状态的问题在于，很难推断出如何在不同的线程、进程和设备上使用和更新它，并且当最终用户隐藏了熵产生和消耗的详细信息时就很容易搞砸。众所周知，`Mersenne Twister PRNG`存在许多问题，它的状态有2.5kb之大，这会导致有问题的初始化。而且它难以通过现代的BigCrush测试，速度很慢。

### JAX PRNG

相反，JAX实现了显示PRNG，其中通过显示传递和迭代PRNG状态来处理熵的产生和消耗。JAX使用可拆分的现代[Threefry counter-based PRNG](https://github.com/google/jax/blob/master/design_notes/prng.md)。也就是说，它的设计使我们能够将PRNG状态分叉到新的PRNG中，以用于并行随机生成。

随机状态由两个我们称为 `key` 的无符号32位整数表示：

In [19]:
from jax import random
key = random.PRNGKey(0)
key

DeviceArray([0, 0], dtype=uint32)

JAX的随机函数会从PRNG状态生成伪随机数，但**不会**更改状态！

重用统一状态将导致**悲伤**和**单调**，最终使终端用户产生混乱：

In [20]:
print(random.normal(key, shape=(1,)))
print(key)
# No no no!
# 别别别！
print(random.normal(key, shape=(1,)))
print(key)

[-0.78476578]
[0 0]
[-0.78476578]
[0 0]


每当我们需要一个新的伪随机数时，我们就将PRNG拆分为一个可用的子密钥：

In [21]:
print("old key", key)
key, subkey = random.split(key)
normal_pseudorandom = random.normal(subkey, shape=(1,))
print("    \---SPLIT --> new key   ", key)
print("             \--> new subkey", subkey, "--> normal", normal_pseudorandom)

old key [0 0]
    \---SPLIT --> new key    [4146024105  967050713]
             \--> new subkey [2718843009 1272950319] --> normal [0.1930774]


每当需要新的随机数时，我们都会传播**key**并创建新的**subkey**：

In [22]:
print("old key", key)
key, subkey = random.split(key)
normal_pseudorandom = random.normal(subkey, shape=(1,))
print("    \---SPLIT --> new key   ", key)
print("             \--> new subkey", subkey, "--> normal", normal_pseudorandom)

old key [4146024105  967050713]
    \---SPLIT --> new key    [2384771982 3928867769]
             \--> new subkey [1278412471 2182328957] --> normal [0.00870719]


我们也可以一次生成多个子键：

In [23]:
key, *subkeys = random.split(key, 4)
for subkey in subkeys:
    print(random.normal(subkey, shape=(1,)))

[-1.17937575]
[-0.14168412]
[1.01073777]


## 🔪 控制流

### ✔ Python控制流 + 自动微分✔

如果只想将`grad`应用于Python函数，毫无问题可以使用常规的Python控制流，就像使用[Autograd](https://github.com/hips/autograd)（或PyTorch和TF Eager）一样。

In [24]:
def f(x):
      if x < 3:
        return 3. * x ** 2
      else:
        return -4 * x

print(grad(f)(2.))  # ok!
print(grad(f)(4.))  # ok!

12.0
-4.0


### Python控制流 + JIT

通过 `jit`使控制流更加复杂，并且默认情况下它具有更多约束。

这样可行：


In [25]:
@jit 
def f(x):
    for i in range(3):
        x = 2 * x
    return x

print(f(3))

24


这样也行:

In [26]:
@jit
def g(x):
    y = 0
    for i in range(x.shape[0]):
        y = y + x[i]
    return y

print(g(jnp.array([1., 2., 3.])))

6.0


但这样至少在默认情况下不行：

In [27]:
@jit
def f(x):
    if x < 3:
        return 3. * x ** 2
    else:
        return -4 * x
    
# 这样会报错

try:
    f(2)
except Exception as e:
    print("Exception {}".format(e))
    

Exception Abstract tracer value encountered where concrete value is expected: Traced<ShapedArray(bool[], weak_type=True)>with<DynamicJaxprTrace(level=0/1)>
The problem arose with the `bool` function. 
While tracing the function f at <ipython-input-27-f8b956c895f5>:1, this concrete value was not available in Python because it depends on the value of the arguments to f at <ipython-input-27-f8b956c895f5>:1 at flattened positions [0], and the computation of these values is being staged out (that is, delayed rather than executed eagerly).
 (https://jax.readthedocs.io/en/latest/errors.html#jax._src.errors.ConcretizationTypeError)


**发生啥事！？**

当我们对一个函数进行jit编译时，我们通常希望编译一个可用于许多不同参数值的函数版本，以便我们可以缓存和重用已编译的代码。这样的话我们不必在每次功能验证时都重新编译。

例如，如果我们在数组 `jnp.array([1., 2., 3.], jnp.float32)` 上验证一个 `jit`函数，我们可能要编译可重用的代码，例如`jnp.array([4., 5., 6.])`，这样可以节省编译时间。

为了获得对许多不同参数值有效地Python代码视图，JAX会在代表可能输入集的抽象集上跟踪它。存在多个不同的抽象级别，不同的变换使用不同的抽象级别。

默认情况下，jit在 `ShapedArray`抽象级别上跟踪您的代码，其中每个抽象值代表具有固定形状和数据类型的所有数值组的集合。例如，如果我们使用抽象值 `ShapedArray((3.,), jnp.float32)`来跟踪，则将获得可以在相应数组集中用于任何具体值的函数视图，这样我们就可以节省编译时间。

但这里需要权衡一下：如果我们在一个不承诺特定具体值的`ShapedArray((), jnp.float32)`上跟踪Python函数，则当我们碰到 `x<3`这样的代码时，表达式`x<3`计算得出一个抽象 `ShapedArray((), jnp.bool_)`，它表示集合 `{True, False}`。当Python尝试将其强制转换为具体的 `True` 或 `False`时，我们会收到错误消息：我们不知道采用哪个分支，也无法继续跟踪！而折中方案是，使用更高级别的抽象，我们可以获得更通用的Python代码视图（从而节省了重新编译的时间），但我们需要对Python代码加入更多的约束来完成跟踪。

好消息是，您可以自己控制这种这种。通过让jit跟踪更精细的抽象值，您可以放款可追溯性约束。例如，对jit使用 `static_argnums` 参数，我们可以指定跟踪某些参数的具体值。以下是该示例函数：

In [28]:
def f(x):
      if x < 3:
        return 3. * x ** 2
      else:
        return -4 * x

f = jit(f, static_argnums=(0,))

print(f(2.))

12.0


另外一个例子，包括了一个循环：

In [29]:
def f(x, n):
    y = 0.
    for i in range(n):
        y = y + x[i]
    return y

f = jit(f, static_argnums=(1, ))

f(jnp.array([2., 3., 4.]), 2)

DeviceArray(5., dtype=float64)

实际上，循环式静态展开的。JAX还可以跟踪更高级别的抽象，例如 `Unshaped`， 但目前这并不被任何变换默认使用。

### ️⚠️ 具有参数值依赖形状的函数

一些控制流问题可能会以更微秒的情况出现：我们要即时编译的数值函数不能讲内部数组的形状专门用于参数值（可以对参数形状进行专门化处理）。举个简单的例子，让我们创建一个函数，其输出恰好取决于输入变量的长度。

In [30]:
def example_fun(length, val):
    return jnp.ones((length, )) * val

# 不jit的情况下没问题
print(example_fun(5, 4))

bad_example_jit = jit(example_fun)
# 这样会报错
try:
    print(bad_example_jit(10, 4))
except Exception as e:
    print("Exception {}".format(e))
    
# `static_argnums`告诉JAX根据这些参数位置重编译
good_example_jit = jit(example_fun, static_argnums=(0, ))
# 第一次编译
print(good_example_jit(10, 4))
# 重编译
print(good_example_jit(5, 4))

[4. 4. 4. 4. 4.]
Exception The numpy.ndarray conversion method __array__() was called on the JAX Tracer object Traced<ShapedArray(int64[], weak_type=True)>with<DynamicJaxprTrace(level=0/1)> (https://jax.readthedocs.io/en/latest/errors.html#jax._src.errors.TracerArrayConversionError)
[4. 4. 4. 4. 4. 4. 4. 4. 4. 4.]
[4. 4. 4. 4. 4.]


如果我们示例中的`length`很少改变，那么`static_argnums` 会很方便，但如果非常多变，那么可能结果会是灾难性的！

最后，如果您的函数具有全局副作用，那么JAX的跟踪器可能会导致发生奇怪的事情。一个常见的情景是尝试在jit函数中打印数组：

In [31]:
@jit
def f(x):
  print(x)
  y = 2 * x
  print(y)
  return y
f(2)

Traced<ShapedArray(int64[], weak_type=True)>with<DynamicJaxprTrace(level=0/1)>
Traced<ShapedArray(int64[], weak_type=True)>with<DynamicJaxprTrace(level=0/1)>


DeviceArray(4, dtype=int64)

### 结构化控制流原语

JAX有更多控制流选项。假设您要避免重新编译，但仍要使用可追溯的控制流，这样可以避免展开大循环。您可以使用以下4中结构化的控制流原语：

* `lax.cond` 可微
* `lax.while_loop` 前向模式可微
* `lax.fori_loop` 前向模式可微
* `lax.scan` 可微

#### cond

等价的Python：

In [32]:
def cond(pred, true_fun, false_fun, operand):
      if pred:
        return true_fun(operand)
      else:
        return false_fun(operand)

In [33]:
from jax import lax

operand = jnp.array([0.])
lax.cond(True, lambda x: x+1, lambda x: x-1, operand)
# --> array([1.], dtype=float32)
lax.cond(False, lambda x: x+1, lambda x: x-1, operand)
# --> array([-1.], dtype=float32)

DeviceArray([-1.], dtype=float64)

#### while_loop

等价的Python：

In [34]:
def while_loop(cond_fun, body_fun, init_val):
    val = init_val
    while cond_fun(val):
        val = body_fun(val)
    return val

In [35]:
init_val = 0
cond_fun = lambda x: x<10
body_fun = lambda x: x+1
lax.while_loop(cond_fun, body_fun, init_val)
# --> array(10, dtype=int32)

DeviceArray(10, dtype=int64)

#### fori_loop

等价的Python：

In [36]:
def fori_loop(start, stop, body_fun, init_val):
    val = init_val
    for i in range(start, stop):
        val = body_fun(i, val)
    return val

In [37]:
init_val = 0
start = 0
stop = 10
body_fun = lambda i,x: x+i
lax.fori_loop(start, stop, body_fun, init_val)
# --> array(45, dtype=int32)

DeviceArray(45, dtype=int64)

#### 总结

| construct      | jit | grad |
|----------------|-----|------|
| if             | ×   | √    |
| for            | √*  | √    |
| while          | √*  | √    |
| lax.cond       | √   | √    |
| lax.while_loop | √   | fwd  |
| lax.fori_loop  | √   | fwd  |
| lax.scan       | √   | √    |

$\ast$ = 与参数**值**无关的循环条件-展开循环

## 🔪 NaNs

### 调试NaNs

如果要跟踪函数或梯度中NaNs发生的位置，可以通过以下方法打开NaNs检查器：

* 在环境变量中设置 `JAX_DEBUG_NANS=True`
* 在主文件上方加入 `from jax.config import config`与 `config.update("jax_debug_nans", True)`
* 在主文件上方加入 `from jax.config import config`和 `config.parse_flags_with_absl()`，之后在命令行选项中加入 `--jax_debug_nans=True`

这样将会在计算生成NaNs的时候立即出错。启用此选项会为XLA生成的每个浮点类型值添加一个NaN教研。这意味着将值送回本机，并针对不在 `@jit` 下的每个原始操作将其作为 `ndarray`进行检查。对于 `@jit` 下的代码，将检查每个 `@jit` 函数的输出，如果存在NaNs，它将在非优化的逐个操作模式下重新运行该函数，从而一次有效地删除一一个级别的 `@jit`。

不过可能会出现一些棘手的情况，例如仅在 `@jit` 下发生的NaNs而不是在费优化模式下产生的。在这种情况下，您会看到一条警告消息，但您的代码继续执行。

如果在梯度求值的后向传递中生成NaNs，则在堆栈跟踪中向上引发几帧异常时，您将使用后想传递函数，该函数本质上是一个简单的jaxpr解释器，用于遍历原始操作的序列。在下面的示例中，我们使用命令行 `JAX_DEBUG_NANS=True ipython`启动了一个ipython repl，然后运行：

In [38]:
import jax.numpy as jnp
jnp.divide(0., 0.)

FloatingPointError: invalid value (nan) encountered in div

生成的NaN被捕获到了。通过运行 `%debug`，我们可以获取事后调试器。这也可以与`@jit`下的函数一起使用，如下所示：

In [39]:
from jax import jit
# %debug
@jit
def f(x, y):
    a = x * y
    b = (x + y) / (x - y)
    c = a + 2
    return a + b * c

x = jnp.array([2., 0.])
y = jnp.array([3., 0.])
f (x, y)

Invalid value encountered in the output of a jit function. Calling the de-optimized version.


FloatingPointError: invalid value (nan) encountered in div

当此代码在@jit函数的输出中看到NaN是，他会调用未优化的代码，因此我们仍然可以获得清晰的堆栈跟踪。然后，我们可以使用 `%debug` 运行事后调试器，以检查所有值以找出错误。

⚠️如果不进行调试，则不应该启用NaN检查器，因为它会引入很多设备主机往返和性能下降的问题！

### 双精度（64位）

目前，JAX默认情况下会强制使用单精度数字，以缓解NumPy API积极将操作数提高一倍的趋势。这是许多机器学应用程序所期望的行为，但这样可能会让您出乎意料。


In [40]:
x = random.uniform(random.PRNGKey(0), (1000,), dtype=jnp.float64)
x.dtype

dtype('float64')

要使用双精度数字，您需要在启动时设置`jax_enable_x64`配置变量。

有几种方法可以开启：

* 直接设置环境变量 `JAX_ENABLE_X64=True`
* 在开始时手动设置`jax_enable_x64`

In [42]:
from jax.config import config
config.update("jax_enable_x64", True)

* 通过 `absl.app.run(main)`来解析

In [ ]:
from jax.config import config
config.config_with_absl()

* 如果您希望JAX为您运行absl解析，即您不想执行`absl.app.run（main）`，则可以改用：

In [41]:
from jax.config import config
if __name__ == '__main__':
  # calls config.config_with_absl() *and* runs absl parsing
  config.parse_flags_with_absl()

请注意，＃2-＃4适用于JAX的任何配置选项。

然后，我们可以确认启用了x64模式：


### 注意事项

⚠️XLA并非在所有后端上都支持64位卷积！

## 结语

